In [ ]:
!pip install --upgrade google-cloud-aiplatform
!pip install pinecone-client
import sys
!{sys.executable} -m pip install langchain huggingface_hub
!{sys.executable} -m  pip install bitsandbytes>=0.39.0
!{sys.executable} -m pip install --upgrade accelerate
!{sys.executable} -m pip install --upgrade sentence_transformers
!{sys.executable} -m  pip install --upgrade transformers
!{sys.executable} -m pip install --upgrade trl
!{sys.executable} -m pip install --upgrade peft
!{sys.executable} -m pip install --upgrade sqlite3
!{sys.executable} -m pip install --upgrade pytz
!{sys.executable} -m pip install --upgrade pypdf PyPDF2
!{sys.executable} -m pip install --upgrade langchain_experimental
!{sys.executable} -m pip install --upgrade langchain_community
!{sys.executable} -m pip install --upgrade langchain_openai
!{sys.executable} -m pip install --upgrade openai tiktoken chromadb
!pip install unstructured['pdf'] unstructured
!{sys.executable} -m pip install --upgrade textwrap
!pip install --upgrade --quiet  cohere
!pip install langchain-cohere
!pip install grandalf
!pip install wikipedia
!pip install faiss-cpu
!pip install faiss-gpu
!pip install langchainhub
!pip install -qU langchain-mistralai
!pip install "unstructured[html]"
!pip install bs4
!pip install rank_bm25
!pip install gradio
!pip install langchain_chroma


In [ ]:
import gradio as gr
from huggingface_hub import InferenceClient

import os
import time
import sqlite3
import re
from pytz import timezone
import pytz
import datetime
import json
import textwrap
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
from operator import itemgetter
import os
import sys

import peft
import bitsandbytes
import accelerate
import trl
from accelerate import init_empty_weights
import torch
import pandas as pd

from transformers import AutoConfig, BitsAndBytesConfig
from accelerate.utils import BnbQuantizationConfig, load_and_quantize_model
from accelerate import load_checkpoint_and_dispatch
import accelerate, bitsandbytes

from langchain_mistralai import ChatMistralAI
import getpass
from google.colab import userdata

import cohere

from langchain import hub
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain_experimental.agents import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader, PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.document_loaders import BSHTMLLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain.llms import HuggingFaceHub
from langchain_community.llms import Cohere
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.callbacks.tracers import ConsoleCallbackHandler

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentType, initialize_agent
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage, AIMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain import PromptTemplate, FewShotPromptTemplate

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.documents import Document


from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

from pinecone import Pinecone, ServerlessSpec



**Api keys setup**

In [ ]:
Mistral_TOKEN = userdata.get('mistralapi')
os.environ['MISTRAL_API_KEY'] = Mistral_TOKEN

co = cohere.Client(coherekey)
os.environ["COHERE_API_KEY"] = coherekey


pckey = userdata.get('pinecone')
pc = Pinecone(api_key=pckey)

**Instantiate your pinecone index**

In [ ]:
index_name = "rag"
index = pc.Index(index_name)
index.describe_index_stats()

**Embeddings model**

In [ ]:

model_name = "Alibaba-NLP/gte-large-en-v1.5"
model_kwargs = {'device': 'cpu', 'trust_remote_code': True}
encode_kwargs = {'normalize_embeddings': False}
### NOTE dim size is 1024 NOT 768 ########
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

**Pinecone retriever**

In [ ]:
def pineconeretriever(query, top_k=12, namespace='ALTEON_CLI_GUIDE', returnLCDocument=True):

  contexts = []
  citations = []

  if not query:
      return ["No question was submitted so no context was retrieved"],['No citations']
  question = query


  res = index.query(namespace=namespace,vector=hf.embed_query(question), top_k=top_k, include_metadata = True )
  contexts = contexts + [
      Document(page_content = x['metadata']['text'], metadata ={'source': x['metadata']['source'] }) if returnLCDocument else x['metadata']['text'] for x in res['matches']
  ]
  citations = citations + [
      x['metadata']['source'] for x in res['matches']
  ]
  return contexts, citations

**Mistral used as the LLM here**

In [ ]:
llmMist = ChatMistralAI(model="mistral-large-latest", temperature =0)

**Cohere Reranker**

In [ ]:
def do_coherererank( query ='', documents =[],top_n = 5,returnLCDocument= True):

  if len(documents) == 0:
      return []
  docs = [
      doc.page_content if isinstance(doc, Document) else doc for doc in documents
  ]
  results = co.rerank(query=query, documents=docs, top_n= top_n, model='rerank-english-v3.0', return_documents= False)
  if hasattr(results, "results"):
      results = getattr(results, "results") # returns list of results
  result_dicts = []
  for res in results:
      result_dicts.append(
          {"index": res.index, "relevance_score": res.relevance_score}
      )

  compressed = []

  for i in result_dicts:
      doc_copy = docs[i["index"]]
      if returnLCDocument: # convert to Document type
        doc_copy = Document(page_content=doc_copy, metadata=documents[i["index"]].metadata) if isinstance(documents[0], Document) else Document(page_content=doc_copy)
        doc_copy.metadata["relevance_score"] = i["relevance_score"]
        doc_copy.metadata["order_idx"] = i["index"]
      compressed.append(doc_copy)

  return compressed

**RAG pipeline and QA**

In [ ]:
##### template ####

template = """
Answer the following question based only on the provided context:

<context>

{context}

</context>

Question: {query}"""

prompt = PromptTemplate(
    input_variables=["context", "query"],
    template=template,
)


def format_chunks(docs):
  """
  Format the chunks for the prompt context
  """
  return "\n\n".join(doc.page_content for doc in docs) if isinstance(docs[0], Document) else "\n\n".join(docs)

query ='How to disable management access to data ports ?'
print ("query: ", query)

### vector retrieval
context, citations = pineconeretriever(query, top_k=12, namespace='ALTEON_CLI_GUIDE', returnLCDocument=True)

##### ensemble retrieval
retrieverbm25 = BM25Retriever.from_texts(["Arnab says to enable data port with management access use /c/sys/ports/ena"])
retrieverbm25.k = 4
ensemble_retriever_cliguide = EnsembleRetriever(retrievers=[RunnableLambda(lambda query: context), retrieverbm25 ],weights=[0.8, 0.2])
ensemblecontext = ensemble_retriever_cliguide.invoke(query)

### rerank
compressed_context = do_coherererank( query =query, documents = ensemblecontext,top_n = 5,returnLCDocument= True)

#### fill prompt with query and compressed context
filledprompt = prompt.format(context=format_chunks(compressed_context), query=query)


#### call mistral llm
response = llmMist.invoke(filledprompt)
print ("response", response.content)




